In [1]:
# !pip install pyspike

In [2]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

import os

import numpy as np

%cd ../scripts/

/home/drfrbc/Neural-Modeling/scripts


In [29]:
from scripts.compare_sta import get_all_directories_within, group_directories_by_cell_and_seed
sim_directory = '11-06-2024-11-56-10-STAs/'
sim_directories = get_all_directories_within(sim_directory)
grouped_directories = group_directories_by_cell_and_seed(sim_directories)

In [41]:
import analysis

from __future__ import print_function
import matplotlib.pyplot as plt
import pyspike as spk

def compare_two_sims(sim_directory1: str, sim_directory2: str):
    parameters = analysis.DataReader.load_parameters(sim_directory1)
    spike_trains = [spk.SpikeTrain(spike_times=analysis.DataReader.read_data(sim_directory1, "soma_spikes"), edges = parameters.h_tstop), 
                spk.SpikeTrain(spike_times=analysis.DataReader.read_data(sim_directory2, "soma_spikes"), edges = parameters.h_tstop)]

    f = spk.spike_sync_profile(spike_trains[0], spike_trains[1])

    return f.avrg()

def compare_simulations(sim_dict):
    overall_results = {}
    for reduced_type in ['Trees', 'Branches']:
        results = {}
        for seed in sim_dict['Complex']:
            complex_dir = sim_dict['Complex'][seed]
            reduced_dir = sim_dict[reduced_type][seed]
            val = compare_two_sims(complex_dir, reduced_dir)
            results[seed] = val

        # Extract values from the results dictionary
        values = list(results.values())

        # Calculate mean and standard deviation
        mean_value = round(np.mean(values), 3)
        std_value = round(np.std(values), 3)

        print(f"{reduced_type} spike synchrony: (mean: {mean_value}, std: {std_value})")
        
        overall_results[reduced_type] = results
        
    return overall_results

In [44]:
results = compare_simulations(grouped_directories)

Trees spike synchrony: (mean: 0.664, std: 0.085)
Branches spike synchrony: (mean: 0.715, std: 0.071)


In [46]:
results

{'Trees': {'Np5': 0.7582938388625592,
  'Np10': 0.5274725274725275,
  'Np1000': 0.6746825182383139,
  'Np1000000': 0.6963746223564955},
 'Branches': {'Np5': 0.7161458333333334,
  'Np10': 0.7562614538790471,
  'Np1000': 0.7867636571102553,
  'Np1000000': 0.6004765687053217}}

Some more examples from pyspike

In [ ]:
# spike_trains = spk.load_spike_trains_from_txt("PySpike_testdata.txt",#"SPIKE_Sync_Test.txt",
#                                               edges=(0, 4000))

In [ ]:
# def compare_two_sims(sim_directory1: str, sim_directory2: str):
#     parameters = analysis.DataReader.load_parameters(sim_directory1)
#     spike_trains = [spk.SpikeTrain(spike_times=analysis.DataReader.read_data(sim_directory1, "soma_spikes"), edges = parameters.h_tstop), 
#                 spk.SpikeTrain(spike_times=analysis.DataReader.read_data(sim_directory2, "soma_spikes"), edges = parameters.h_tstop)]
    
#     # plt.figure()

#     # f = spk.spike_sync_profile(spike_trains[0], spike_trains[1])
#     f = spk.spike_sync_profile(spike_trains[0], spike_trains[1])
#     # x, y = f.get_plottable_data()
#     # plt.plot(x, y, '--ok', label="SPIKE-SYNC profile")
#     # print(f.x)
#     # print(f.y)
#     # print(f.mp)

#     # print(sim_directory1, sim_directory2)
#     # print("Average:", f.avrg())


#     # f = spk.spike_profile(spike_trains[0], spike_trains[1])
#     # x, y = f.get_plottable_data()

#     # plt.plot(x, y, '-b', label="SPIKE-profile")

#     # plt.axis([0, 4000, -0.1, 1.1])
#     # plt.legend(loc="center right")

#     # plt.figure()

#     # plt.subplot(211)

#     # f = spk.spike_sync_profile(spike_trains)
#     # x, y = f.get_plottable_data()
#     # plt.plot(x, y, '-b', alpha=0.7, label="SPIKE-Sync profile")

#     # x1, y1 = f.get_plottable_data(averaging_window_size=50)
#     # plt.plot(x1, y1, '-k', lw=2.5, label="averaged SPIKE-Sync profile")

#     # plt.subplot(212)

#     # f_psth = spk.psth(spike_trains, bin_size=50.0)
#     # x, y = f_psth.get_plottable_data()
#     # plt.plot(x, y, '-k', alpha=1.0, label="PSTH")


#     # print("Average:", f.avrg())

#     # plt.show()

In [ ]:
# plt.figure()
# isi_distance = spk.isi_distance_matrix(spike_trains)
# plt.imshow(isi_distance, interpolation='none')
# plt.title("ISI-distance")

# plt.figure()
# spike_distance = spk.spike_distance_matrix(spike_trains, interval=(0, 1000))
# plt.imshow(spike_distance, interpolation='none')
# plt.title("SPIKE-distance, T=0-1000")

# plt.figure()
# spike_sync = spk.spike_sync_matrix(spike_trains, interval=(2000, 4000))
# plt.imshow(spike_sync, interpolation='none')
# plt.title("SPIKE-Sync, T=2000-4000")

# plt.show()

In [ ]:
# from __future__ import print_function

# import pyspike as spk
# from datetime import datetime
# import cProfile
# import pstats

# # in case you dont have the cython backends, disable the warnings as follows:
# # spk.disable_backend_warning = True

# M = 100    # number of spike trains
# r = 1.0    # rate of Poisson spike times
# T = 1E3    # length of spike trains

# print("%d spike trains with %d spikes" % (M, int(r*T)))

# spike_trains = []

# t_start = datetime.now()
# for i in range(M):
#     spike_trains.append(spk.generate_poisson_spikes(r, T))
# t_end = datetime.now()
# runtime = (t_end-t_start).total_seconds()

# sort_by = 'tottime'
# # sort_by = 'cumtime'

# print("Spike generation runtime: %.3fs" % runtime)
# print()

# print("================ ISI COMPUTATIONS ================")
# print("    MULTIVARIATE DISTANCE")
# cProfile.run('spk.isi_distance(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)

# print("    MULTIVARIATE PROFILE")
# cProfile.run('spk.isi_profile(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)

# print("================ SPIKE COMPUTATIONS ================")
# print("    MULTIVARIATE DISTANCE")
# cProfile.run('spk.spike_distance(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)

# print("    MULTIVARIATE PROFILE")
# cProfile.run('spk.spike_profile(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)

# print("================ SPIKE-SYNC COMPUTATIONS ================")
# print("    MULTIVARIATE DISTANCE")
# cProfile.run('spk.spike_sync(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)

# print("    MULTIVARIATE PROFILE")
# cProfile.run('spk.spike_sync_profile(spike_trains)', 'performance.stat')
# p = pstats.Stats('performance.stat')
# p.strip_dirs().sort_stats(sort_by).print_stats(5)